In [1]:
%load_ext autoreload
%autoreload 2
from app.config_loader import load_config_once
conf = load_config_once(output_folder="SIDKIM")
import pdfplumber, camelot, pprint, json5,json, os, re
import fitz #type: ignore
import pandas as pd
import numpy as np

from app.fund_sid_data import *
from app.utils import *

with open("config\\sid_params.json5","r+") as file:
    params = json5.load(file)

# with open("config\\sid_params.json5","r+") as file:
#     params = json5.dump(params,file)
# | Tool      | Page Indexing | Example                  |
# | --------- | ------------- | ------------------------ |
# | `fitz`    | **0-based**   | `doc[0]` → first page    |
# | `camelot` | **1-based**   | `pages="1"` → first page |

Loaded config with output_folder = SIDKIM


In [2]:
path = r"C:\Users\Kaustubh.keny\Projects\PDF\SID\Apr"
sid_fund_apr = Helper.get_pdf_paths(path)
path = r"C:\Users\Kaustubh.keny\Projects\PDF\SID\Mar"
sid_fund_mar = Helper.get_pdf_paths(path)

In [4]:
sid_fund_mar

{'360 One Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Mar\\360 One Mutual Fund\\18_16905_Mar-2025_1741330917_SID.pdf',
 'Aditya Birla Sun Life Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Mar\\Aditya Birla Sun Life Mutual Fund\\3_16940_Mar-2025_1741838999_SID.pdf',
 'Angel One Mututal Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Mar\\Angel One Mututal Fund\\96_16971_Mar-2025_1742355460_SID.pdf',
 'Angel One Mututal Fund_B': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Mar\\Angel One Mututal Fund\\96_16971_Mar-2025_1742355460_SID_all_ocr.pdf',
 'Axis Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Mar\\Axis Mutual Fund\\1_16909_Mar-2025_1741331175_SID.pdf',
 'Axis Mutual Fund_B': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Mar\\Axis Mutual Fund\\1_16928_Mar-2025_1741758037_SID.pdf',
 'Bandhan Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Mar\\Bandhan Mutual Fund\\16_16899_Mar-2025_1741148354_SID.pdf',
 'Edelweiss

In [3]:
sid_fund_apr

{'Bajaj Finserv Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Apr\\Bajaj Finserv Mutual Fund\\59_17022_Apr-2025_1744773931_SID.pdf',
 'Bajaj Finserv Mutual Fund_B': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Apr\\Bajaj Finserv Mutual Fund\\59_17047_Apr-2025_1745296581_SID.pdf',
 'Canara Robeco Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Apr\\Canara Robeco Mutual Fund\\6_16997_Apr-2025_1743747357_SID.pdf',
 'Dsp Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Apr\\DSP Mutual Fund\\8_17037_Apr-2025_1744864544_SID.pdf',
 'Dsp Mutual Fund_B': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Apr\\DSP Mutual Fund\\8_17039_Apr-2025_1744864628_SID.pdf',
 'Edelweiss Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Apr\\Edelweiss Mutual Fund\\9_17045_Apr-2025_1745209928_SID.pdf',
 'Groww Mutual Fund': 'C:\\Users\\Kaustubh.keny\\Projects\\PDF\\SID\\Apr\\Groww Mutual Fund\\20_17033_Apr-2025_1744864352_SID.pdf',
 'Groww Mutual Fund_B': 'C:\

SID

In [ ]:
path = sid_fund_mar["360 One Mutual Fund"]
object = ThreeSixtyOne(amc_id="18", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero("1"))
temp_dict.update(object.parse_scheme_table_data("5,6,7,8"))
temp_dict.update(object.parse_fund_manager_info("16"))

In [ ]:
path = sid_fund_mar["Angel One Mututal Fund"]
object = AngelOne(amc_id="96", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero("1"))
temp_dict.update(object.parse_scheme_table_data("5,6,7,8"))
temp_dict.update(object.parse_fund_manager_info("14,15"))

In [ ]:
path = sid_fund_apr["Dsp Mutual Fund_B"]
object = DSP(amc_id="8", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero("1"))
temp_dict.update(object.parse_scheme_table_data("5-9"))
temp_dict.update(object.parse_fund_manager_info("22"))


In [ ]:
path = sid_fund_apr["Groww Mutual Fund"]
object = GROWW(amc_id="20", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero("1"))
temp_dict.update(object.parse_scheme_table_data("4,5,6,7"))
temp_dict.update(object.parse_fund_manager_info("27"))

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","Apr SID\\Edelweiss Mutual Fund\\9_17045_Apr-2025_1745209928_SID.pdf")
object = Edelweiss(amc_id="9", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero([0]))
temp_dict.update(object.parse_scheme_table_data("4,5,6,7,8"))
temp_dict.update(object.parse_fund_manager_info("15,16"))
# data = object.refine_extracted_data(temp_dict)
# dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Tata Mutual Fund\\38_17082_May-2025_1746423571_SID.pdf")
object = Tata(amc_id="38", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero([0]))
temp_dict.update(object.parse_scheme_table_data("3,4,5"))
temp_dict.update(object.parse_fund_manager_info("12"))
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Nippon India Mutual Fund\\33_17142_May-2025_1747631690_SID.pdf")
object = Nippon(amc_id="33", path=path)
temp_dict = {}
# temp_dict.update(object.parse_page_zero([0]))
# temp_dict.update(object.parse_scheme_table_data("3,4,5,6,7"))
temp_dict.update(object.parse_fund_manager_info("11"))
# data = object.refine_extracted_data(temp_dict)
# dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

Function Running: _parse_fund_manager_info


c:\Users\rando\OneDrive\Documents\mywork-repo\app\parse_sid_pdf.py:192: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  


In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\SBI Mutual Fund\\35_17126_May-2025_1747282059_SID.pdf")
object = SBI(amc_id="35", path=path)
temp_dict = {}
# temp_dict.update(object.parse_page_zero([0]))
# temp_dict.update(object.parse_scheme_table_data("3,4,5"))
temp_dict.update(object._parse_fund_manager_info("13"))
# data = object.refine_extracted_data(temp_dict)
# dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

Function Running: _parse_fund_manager_info


c:\Users\rando\OneDrive\Documents\mywork-repo\app\parse_sid_pdf.py:192: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  


In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Samco Mutual Fund\\58_17166_May-2025_1748246684_SID.pdf")
object = Samco(amc_id="58", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero([0]))
temp_dict.update(object.parse_scheme_table_data("3,4,5"))
temp_dict.update(object._parse_fund_manager_info("14,15"))
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Baroda BNP Paribas Mutual Fund\\2_17136_May-2025_1747367452_SID.pdf")
object = BarodaBNP(amc_id="2", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero([0]))
temp_dict.update(object.parse_scheme_table_data("3,4,5,6"))
temp_dict.update(object._parse_fund_manager_info("13"))
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Unifi Mutual Fund\\1747199772321.pdf")
object = Unifi(amc_id="97", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero([0]))
temp_dict.update(object.parse_scheme_table_data("5,6,7,8,9,10"))
temp_dict.update(object._parse_fund_manager_info("26,27"))
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

In [ ]:
path = os.path.join(os.getcwd(),"pdfs","May SID\\Motilal Oswal Mutual Fund\\28_17164_May-2025_1748246608_SID.pdf")
object = MotilalOswal(amc_id="28", path=path)
temp_dict = {}
temp_dict.update(object.parse_page_zero([0]))
temp_dict.update(object.parse_scheme_table_data("4,5,6,7,8"))
temp_dict.update(object._parse_fund_manager_info("19,21,22"))
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)

In [76]:
data = object.refine_extracted_data(temp_dict)
dfs = object.merge_and_select_data(data,sid_or_kim="sid",special_func=False)
with open("temp.json","w+") as file:
    json.dump(temp_dict,file)
with open("data.json","w+") as file:
    json.dump(data,file)
with open("dfs.json","w+") as file:
    json.dump(dfs,file)

Function Running: refine_extracted_data
Function Running: merge_and_select_data
